### Model for prediction outcome on credit card applications

In this notebook, we are going to create a supervised machine learning model to predict the outcome of a credit card application given a set of features. 

In [525]:
# Import modules 
import numpy as np 
import pandas as pd

#### 1. Method of dropping rows with missing values
We reload the original credit card dataset and clean the missing data by simply dropping the concerning rows.  

In [526]:
cc_original = pd.read_csv('../datasets/crx.data_named.csv', index_col=[0])
cc_original.head()

,Gender,Age,Debt,Married,BankCustomer,EducationLevel,Ethnicity,YearsEmployed,PriorDefaulter,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,Approved
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [527]:
cc_original.describe(include='all')

,Gender,Age,Debt,Married,BankCustomer,EducationLevel,Ethnicity,YearsEmployed,PriorDefaulter,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,Approved
count,690,690,690.000000,690,690,690,690,690.000000,690,690,690.00000,690,690,690,690.000000,690
unique,3,350,NaN,4,4,15,10,NaN,2,2,NaN,2,3,171,NaN,2
top,b,?,NaN,u,g,c,v,NaN,t,f,NaN,f,g,00000,NaN,-
freq,468,12,NaN,519,519,137,399,NaN,361,395,NaN,374,625,132,NaN,383
mean,NaN,NaN,4.758725,NaN,NaN,NaN,NaN,2.223406,NaN,NaN,2.40000,NaN,NaN,NaN,1017.385507,NaN
std,NaN,NaN,4.978163,NaN,NaN,NaN,NaN,3.346513,NaN,NaN,4.86294,NaN,NaN,NaN,5210.102598,NaN
min,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.00000,NaN,NaN,NaN,0.000000,NaN
25%,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,0.165000,NaN,NaN,0.00000,NaN,NaN,NaN,0.000000,NaN
50%,NaN,NaN,2.750000,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,0.00000,NaN,NaN,NaN,5.000000,NaN
75%,NaN,NaN,7.207500,NaN,NaN,NaN,NaN,2.625000,NaN,NaN,3.00000,NaN,NaN,NaN,395.500000,NaN


In [876]:
# Load helper functions from src/utils-*-*.py to run on jupyter-notebook
%run ../src/utils-gather-assess.py
%run ../src/utils-explore-clean.py
%run ../src/utils-models.py

In [529]:
cc_drop = cc_original.copy()

# Replace all ? with Nan values
for icol in cc_drop.columns:
    replace_feature_missingvalues(cc_drop, icol, '?', np.NaN)

In [530]:
# Drop all rows that are Nan
cc_drop.dropna(inplace=True)

# change datatype of Age feature
change_feature_datatype(cc_drop, 'Age', float)

 Details of column: Age
        - dtype(o): object
        - dtype(n): float64


In [531]:
cc_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 653 entries, 0 to 689
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gender          653 non-null    object 
 1   Age             653 non-null    float64
 2   Debt            653 non-null    float64
 3   Married         653 non-null    object 
 4   BankCustomer    653 non-null    object 
 5   EducationLevel  653 non-null    object 
 6   Ethnicity       653 non-null    object 
 7   YearsEmployed   653 non-null    float64
 8   PriorDefaulter  653 non-null    object 
 9   Employed        653 non-null    object 
 10  CreditScore     653 non-null    int64  
 11  DriversLicense  653 non-null    object 
 12  Citizen         653 non-null    object 
 13  ZipCode         653 non-null    object 
 14  Income          653 non-null    int64  
 15  Approved        653 non-null    object 
dtypes: float64(3), int64(2), object(11)
memory usage: 86.7+ KB


After dropping rows with missing values, we end up with 653 rows/instances for 15 features and 1 target variable.  

In [532]:
cc_drop.to_csv('../datasets/crx.data_drop.csv')

In [533]:
show_files_datasets('../datasets')

 datasets/
       - crx.data
       - crx.names
       - crx.data_named.csv
       - crx.data_drop.csv
       - crx.data_clean.csv


Credit card data is now loaded into a `cc_drop` dataframe. We now split the combined dataframe into `features` and `target` dataframe by performing **one_hot_encoding** on categorical variables in `features` dataframe.   

In [534]:
# Split dataframe into num_df, cat_df and target_df
cc_drop_num, cc_drop_cat, cc_drop_target = split_dataframe_datatypes(cc_drop, 'Approved')

/home/naneen/.local/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [535]:
# Perform Ordinal Encoding on Categorical features
print('Before:\n',cc_drop_cat.head())
cc_drop_cat_nparray = apply_encoder_ordinal(cc_drop_cat)
print('\nAfter:\n',cc_drop_cat_nparray)

Before:
   Gender Married BankCustomer EducationLevel Ethnicity PriorDefaulter  \
0      b       u            g              w         v              t   
1      a       u            g              q         h              t   
2      a       u            g              q         h              t   
3      b       u            g              w         v              t   
4      b       u            g              w         v              t   

  Employed DriversLicense Citizen ZipCode  
0        t              f       g   00202  
1        t              f       g   00043  
2        f              f       g   00280  
3        t              t       g   00100  
4        f              f       s   00120  

After:
 [[ 1.  1.  0. ...  0.  0. 68.]
 [ 0.  1.  0. ...  0.  0. 11.]
 [ 0.  1.  0. ...  0.  0. 94.]
 ...
 [ 0.  2.  2. ...  1.  0. 67.]
 [ 1.  1.  0. ...  0.  0. 94.]
 [ 1.  1.  0. ...  1.  0.  0.]]


In [536]:
# Convert df into numpy array --> Numerical variables
print('Before:\n',cc_drop_num.head())
cc_drop_num_nparray = cc_drop_num.to_numpy()
print('\nAfter:\n',cc_drop_num_nparray)

Before:
      Age   Debt  YearsEmployed  CreditScore  Income
0  30.83  0.000           1.25            1       0
1  58.67  4.460           3.04            6     560
2  24.50  0.500           1.50            0     824
3  27.83  1.540           3.75            5       3
4  20.17  5.625           1.71            0       0

After:
 [[3.083e+01 0.000e+00 1.250e+00 1.000e+00 0.000e+00]
 [5.867e+01 4.460e+00 3.040e+00 6.000e+00 5.600e+02]
 [2.450e+01 5.000e-01 1.500e+00 0.000e+00 8.240e+02]
 ...
 [2.525e+01 1.350e+01 2.000e+00 1.000e+00 1.000e+00]
 [1.792e+01 2.050e-01 4.000e-02 0.000e+00 7.500e+02]
 [3.500e+01 3.375e+00 8.290e+00 0.000e+00 0.000e+00]]


Often we (atleast me!) forgets about the categorical target variable. This may result in model giving Nan results on the metrics you are tracking. Let's not do that this time around! 

In [537]:
# Perform Label Encoding on Categorical target variable
print('Before:\n', cc_drop_target['Approved'])
cc_drop_target_nparray = apply_encoder_label(cc_drop_target['Approved'])
print('\nAfter:\n',cc_drop_target_nparray)

Before:
 0      +
1      +
2      +
3      +
4      +
      ..
685    -
686    -
687    -
688    -
689    -
Name: Approved, Length: 653, dtype: object

After:
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 

In [538]:
# Combined features np.array
cc_drop_combined_nparray = combine_two_nparrays(cc_drop_cat_nparray, cc_drop_num_nparray)
cc_drop_combined_nparray.shape

(653, 15)

Let's sneak into the shapes of dataframes that are now ready to go into supervised ML model.

In [539]:
# Check size of features npa
cc_drop_combined_nparray.shape

(653, 15)

In [540]:
# Check size of target npa
cc_drop_target_nparray.shape

(653,)

Before starting to build a ML model, we first initialize model list and dicts.

In [844]:
# Set the no. of processor 
nproc = 2

In [864]:
# initialize models list and dicts
models = []
mean_mse = {}
cv_std = {}
res = {}

In [865]:
# Classical linear regressors
models_clinear  = get_regressors_classical('all')
models_variable = get_regressors_variable('all')
models_bayesian = get_regressors_bayesian('all')

# Update models list
models.extend(models_clinear)
models.extend(models_variable)
models.extend(models_bayesian)

# List of models to perform cross-validation on.
print('Model list for cross-validation:\n')
for imodel, model in enumerate(models):
    print(' {} {}'.format(imodel,model))

Model list for cross-validation:

 0 LinearRegression()
 1 Ridge(alpha=0.5)
 2 RidgeCV(alphas=array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]))
 3 ElasticNet()
 4 Lars()
 5 Lasso()
 6 LassoLars()
 7 OrthogonalMatchingPursuit()
 8 ARDRegression()
 9 BayesianRidge()


In [866]:
# Split dataframe into training and testing datasets
ftrain, ftest, ttrain, ttest = get_train_test_dfs(cc_drop_combined_nparray, cc_drop_target_nparray, 0.7)

In [867]:
# Looking at shapes of training and testing datasets
print(' Train datatset: \n Shape of features:',ftrain.shape, '==> Shape of target:',ttrain.shape)

print('\n Test_ datatset: \n Shape of features:',ftest.shape, '==> Shape of target:',ttest.shape)

 Train datatset: 
 Shape of features: (457, 15) ==> Shape of target: (457,)

 Test_ datatset: 
 Shape of features: (196, 15) ==> Shape of target: (196,)


In [868]:
# Cross validation to predict feature importance  
models_scores = {}
for model in models:
    scores = train_model(model, ftrain, ttrain, nproc, 5, False)
    models_scores[str(model)] = scores

In [869]:
# Print models summary table
print_models_metrics_scores(models_scores)



--------------------------------------------------------------------------------

 Model:  LinearRegression()
     Metric            Mean  
 - fit_time     :     0.00125
 - score_time   :     0.00108
 - test_mse     :    -0.09703
 - test_mae     :    -0.20360
 - test_r2s     :     0.60645

 Model:  Ridge(alpha=0.5)
     Metric            Mean  
 - fit_time     :     0.00124
 - score_time   :     0.00112
 - test_mse     :    -0.09708
 - test_mae     :    -0.20441
 - test_r2s     :     0.60622

 Model:  RidgeCV(alphas=array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]))
     Metric            Mean  
 - fit_time     :     0.00268
 - score_time   :     0.00111
 - test_mse     :    -0.09707
 - test_mae     :    -0.20403
 - test_r2s     :     0.60628

 Model:  ElasticNet()
     Metric            Mean  
 - fit_time     :     0.00116
 - score_time   :     0.00104
 - test_mse     :    -0.22493
 - test_mae     :    -0.44865
 - test_r2s     :     0.08777

 Model:  Lars()
     Metric           

In [877]:
# Print models-metrics-scores in a table
pprint_models_metrics_scores(models_scores)



| Model                | fit_time     | score_time   | test_mse     | test_mae     | test_r2s     |
| -------------------- | ------------ | ------------ | ------------ | ------------ | ------------ |
| LinearRegression()   |   0.00125318 |   0.00107989 |  -0.09702715 |  -0.20359791 |   0.60645284 |
| Ridge(alpha=0.5)     |   0.00123868 |   0.00111885 |  -0.09708125 |  -0.20441315 |   0.60621756 |
| RidgeCV(alphas=array |   0.00268097 |   0.00111103 |  -0.09706838 |  -0.20403166 |   0.60628353 |
| ElasticNet()         |    0.0011579 |   0.00104132 |  -0.22493043 |   -0.4486547 |   0.08776894 |
| Lars()               |   0.00356498 |   0.00084462 |  -0.09637733 |  -0.20319041 |   0.60907186 |
| Lasso()              |   0.00086789 |   0.00085139 |  -0.25541311 |   -0.4808238 |  -0.03555773 |
| LassoLars()          |   0.00096087 |   0.00083814 |  -0.24818679 |  -0.49569393 |  -0.00719922 |
| OrthogonalMatchingPu |   0.00103106 |   0.00077591 |  -0.10037015 |  -0.20009033 |   0.59262991 

This is great! We have now created a readable table for our model-metrics-scores.  This makes it so much easier to visually interpret the models, their metrics and scores.